In [1]:
# # install dependencies
# !pip install -U torch torchvision cython
# !pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# import torch, torchvision
# torch.__version__

# !git clone https://github.com/facebookresearch/detectron2
# !pip install -e detectron2

# from detectron2.config import get_cfg

# !mkdir ./temp
# !rm -rf video.mp4

# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1X0GR6xAT0Jw6XI-y8whn3T-KmDIrRa9n' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1X0GR6xAT0Jw6XI-y8whn3T-KmDIrRa9n" -O video.mp4
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JEkxSn_JDbeg5ezmSkBCDGh6T5zGpLb8' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JEkxSn_JDbeg5ezmSkBCDGh6T5zGpLb8" -O video2.mp4

# !rm -rf ./temp/*
# !ffmpeg -i video.mp4 ./temp/frame%04d.jpg 

# !wget https://dl.fbaipublicfiles.com/densepose/densepose_rcnn_R_50_FPN_s1x/143908701/model_final_dd99d2.pkl
# !mkdir ./processed

# ! git clone https://github.com/erkil1452/gaze360
# ! wget http://gaze360.csail.mit.edu/files/gaze360_model.pth.tar

# !pip install -q lucid>=0.2.3
# !pip install -q moviepy

In [2]:
!rm -rf ./temp/*
!mkdir ./temp
!ffmpeg -i pokemao.mp4 ./temp/frame%04d.jpg 

mkdir: cannot create directory ‘./temp’: File exists
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --e

In [3]:
DENSEPOSE_PATH = '../densepose/detectron2/projects/DensePose/'
import sys
sys.path.append(DENSEPOSE_PATH)

In [4]:
import cv2
import numpy as np
import matplotlib.patches as patches

import matplotlib.pyplot as plt
import cv2
import numpy as np

from typing import ClassVar, Dict

from detectron2.config import get_cfg
from detectron2.structures.instances import Instances
from detectron2.engine.defaults import DefaultPredictor

from densepose import add_densepose_config
from densepose.vis.base import CompoundVisualizer
from densepose.vis.bounding_box import ScoredBoundingBoxVisualizer
from densepose.vis.extractor import CompoundExtractor, create_extractor

from densepose.structures import  DensePoseResult

from densepose.vis.densepose import DensePoseResultsFineSegmentationVisualizer

%matplotlib inline

In [5]:
#UTILS.PY
def mask_to_bbox(mask):
    xs = np.where(np.sum(mask, axis=0) > 0)[0]
    ys = np.where(np.sum(mask, axis=1) > 0)[0]
    
    if len(xs) == 0 or len(ys) == 0:
        return None

    x0 = xs[0]
    x1 = xs[-1]

    y0 = ys[0]
    y1 = ys[-1]
    w = x1-x0
    h = y1-y0
    
    x0 = max(0,x0-w*0.15)
    x1 = max(0,x1+w*0.15)
    y0 = max(0,y0-h*0.15)
    y1 = max(0,y1+h*0.15)

    return np.array((x0, y0, x1, y1), dtype=np.float32)

def draw_bbox(im, bbox, color):
    img = cv2.rectangle(im,(bbox[0],bbox[1]),(bbox[2],bbox[3]),color,20)
    fig = plt.figure(figsize=[12,12])
    plt.imshow(im)


def compute_iou(bb1,bb2):
    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[2]-bb1[0]) * (bb1[3]-bb1[1])
    bb2_area = (bb2[2]-bb2[0]) * (bb2[3]-bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    eps = 1e-8

    if iou <= 0.0  or iou > 1.0 + eps: return 0.0

    return iou


def predictPeople(img, vis, ext):
    outputs = predictor(img)['instances']
    densepose_result = ext(outputs)
    people = []
    for i, result_encoded_w_shape in enumerate(densepose_result.results):
        iuv_arr = DensePoseResult.decode_png_data(*result_encoded_w_shape)
        bbox_xywh = densepose_result.boxes_xywh[i]
        people.append((bbox_xywh, iuv_arr))
    return people

def mergeIUVWithImage(imShape, IUV, bbox):
    x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    new_iuv = np.zeros([3, imShape[0],imShape[1]],dtype=np.uint8)
    new_iuv[: ,y:y+h, x:x+w] = IUV
    return new_iuv

def transposeIUV(IUV):
    iuv0 = IUV[0,:,:]
    iuv1 = IUV[1,:,:]
    iuv2 = IUV[2,:,:]

    return np.concatenate((iuv0[:,:,np.newaxis], iuv1[:,:,np.newaxis], iuv2[:,:,np.newaxis]), axis=2)

In [6]:
cfg = get_cfg()
add_densepose_config(cfg)

cfg.merge_from_file('{}configs/densepose_rcnn_R_50_FPN_s1x.yaml'.format(DENSEPOSE_PATH))
cfg.MODEL.DEVICE = 'cuda'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.WEIGHTS = './model_final_dd99d2.pkl'
predictor = DefaultPredictor(cfg)

Config '../densepose/detectron2/projects/DensePose/configs/densepose_rcnn_R_50_FPN_s1x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [7]:
class Person():
    def __init__(self, bbox, IUV, originalImShape):
        self.bbox = bbox
        self.IUV = IUV
        self.head = None
        
        IUV = mergeIUVWithImage(originalImShape, self.IUV, self.bbox)
        IUV = transposeIUV(IUV)

        mask = (IUV[:,:,0] == 23) + (IUV[:,:,0] == 24)
        if mask.any():
            self.head = mask_to_bbox(mask).astype(np.uint)
        if self.head is not None:
            #TODO: Improve eye location
            self.eyes = [(self.head[0]+self.head[2])/2.0, (0.65*self.head[1]+0.35*self.head[3])]
            
        

In [8]:
import glob
import imageio
import cv2
import numpy as np
import os

filenames = [img for img in glob.glob("./temp/*.jpg")]

filenames.sort() # ADD THIS LINE

segm_vs = DensePoseResultsFineSegmentationVisualizer()
segm_ext = create_extractor(segm_vs)


frames= []
for i, path in enumerate(filenames):
    im = cv2.imread(path)
    predicts = predictPeople(im, segm_vs, segm_ext)
    people = []
    for i in range(0, len(predicts)):
        (bbox, IUV) = predicts[i]
        person = Person(bbox, IUV, im.shape)
        if person.head is None:
            continue
        people.append(person)
    frames.append(people)


In [9]:
def find_id(bbox,id_dict):
    id_final = None
    max_iou = 0.5
    for k in id_dict.keys():
        if(compute_iou(bbox,id_dict[k].head)>max_iou): 
            id_final = k
            max_iou = compute_iou(bbox,id_dict[k].head)
    return id_final

In [10]:
# def find_id(bbox,lastPerson):
#     id_final = None
#     max_iou = 0.5
#     for k in lastPerson.keys():
#         if(compute_iou(bbox,lastPerson[k]['head'])>max_iou): 
#             id_final = k
#             max_iou = compute_iou(bbox,lastPerson[k]['head'])
#     return id_final

In [11]:
def getHeadInFrames(frames):
    id_num = 0
    headInFrames = dict()
    lastPerson = dict()

    for i in range(0, len(frames)):
        people = frames[i]
        if len(people)>0:    
            nextPerson = dict()
            for j in range(len(people)):
                person = people[j]
                if person.head is None: continue
                headId = find_id(person.head,lastPerson)
                if headId is None: 
                    id_num+=1
                    headId = id_num
                nextPerson[headId] = person
            lastPerson = nextPerson
            headInFrames[i] = lastPerson
    return headInFrames

In [12]:
# def get_headInFrames(frames):
#     id_num = 0
#     headInFrames = dict()
#     lastPerson = dict()

#     for i in range(0, len(frames)):
#         people = frames[i]
#         if len(people)>0:    
#             nextPerson = dict()
#             for j in range(len(people)):
#                 person = people[j]
#                 if person.head is None: continue
#                 id_val = find_id(person.head,lastPerson)
#                 if id_val is None: 
#                     id_num+=1
#                     id_val = id_num
#                 nextPerson[id_val] = person.__dict__
#             lastPerson = nextPerson
#             headInFrames[i] = lastPerson
#     return headInFrames

In [13]:
headInFrames = getHeadInFrames(frames)

In [14]:
headInFrames[0]

{1: <__main__.Person at 0x7f9698d5a470>}

In [15]:
import sys
import torch
import torchvision.transforms as transforms
import imageio
import cv2
import random
from PIL import Image
import math
import torch
import torchvision.transforms as transforms
import sys

import numpy as np
import json
import moviepy.editor as mvp
import lucid.misc.io.showing as show
from lucid.misc.gl.glcontext import create_opengl_context
import OpenGL.GL as gl
from OpenGL.GL import shaders

sys.path.append('./gaze360/code/')
from model import GazeLSTM

IMAGE_NORMALIZE = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

class Gaze360:
    def __init__(self, model_path, width, height):
        checkpoint = torch.load(model_path)
        
        self.model = torch.nn.DataParallel(GazeLSTM()).cuda()
        self.model.cuda()
        self.model.load_state_dict(checkpoint['state_dict'])
        self.model.eval()
        
        self.width = width
        self.height = height
        
        create_opengl_context((width, height))
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        vertex_shader_code = open("vertex_shader.txt", "r").read()
        fragment_shader_code = open("fragment_shader.txt", "r").read()

        self.vertexPositions = np.float32([[-1, -1], [1, -1], [-1, 1], [1, 1]])
        VERTEX_SHADER = shaders.compileShader(vertex_shader_code, gl.GL_VERTEX_SHADER)

        FRAGMENT_SHADER = shaders.compileShader(fragment_shader_code, gl.GL_FRAGMENT_SHADER)

        self.shader = shaders.compileProgram(VERTEX_SHADER, FRAGMENT_SHADER)

        self.xpos = gl.glGetUniformLocation(self.shader, 'xpos')
        self.ypos = gl.glGetUniformLocation(self.shader, 'ypos')

        self.vdir_x = gl.glGetUniformLocation(self.shader, 'vdir_x')
        self.vdir_y = gl.glGetUniformLocation(self.shader, 'vdir_y')
        self.vdir_z = gl.glGetUniformLocation(self.shader, 'vdir_z')

        self.arrow_size = gl.glGetUniformLocation(self.shader, 'size')

        self.res_loc = gl.glGetUniformLocation(self.shader, 'iResolution')
        
    def spherical2cartesial(self, x): 
        output = torch.zeros(x.size(0),3)
        output[:,2] = -torch.cos(x[:,1])*torch.cos(x[:,0])
        output[:,0] = torch.cos(x[:,1])*torch.sin(x[:,0])
        output[:,1] = torch.sin(x[:,1])
        return output
    
    def render_frame(self, x_position,y_position,vx,vy,vz,asize):
          gl.glClear(gl.GL_COLOR_BUFFER_BIT)
          with self.shader:

            x_position = x_position*0.89
            y_position = y_position*0.67
            gl.glUniform1f(self.xpos, x_position)
            gl.glUniform1f(self.ypos, y_position)

            gl.glUniform1f(self.vdir_x, vx)
            gl.glUniform1f(self.vdir_y, vy)
            gl.glUniform1f(self.vdir_z, vz)
            gl.glUniform1f(self.arrow_size, asize)

            gl.glUniform3f(self.res_loc, self.width, self.height, 1.0)

            gl.glEnableVertexAttribArray(0);
            gl.glVertexAttribPointer(0, 2, gl.GL_FLOAT, False, 0, self.vertexPositions)
            gl.glDrawArrays(gl.GL_TRIANGLE_STRIP, 0, 4)
          img_buf = gl.glReadPixels(0, 0, self.width, self.height, gl.GL_RGB, gl.GL_UNSIGNED_BYTE)
          img = np.frombuffer(img_buf, np.uint8).reshape(self.height, self.width, 3)[::-1]
          return img

  _np_qint8 = np.dtype([("qint8", np.int8, 1)])

  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

  _np_quint16 = np.dtype([("quint16", np.uint16, 1)])

  _np_qint32 = np.dtype([("qint32", np.int32, 1)])

  np_resource = np.dtype([("resource", np.ubyte, 1)])



In [16]:
import sys
import torch
import torchvision.transforms as transforms
import imageio
import cv2
import random
from PIL import Image
import math
import torch
import torchvision.transforms as transforms
import sys
image_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


sys.path.append('./gaze360/code/')
from model import GazeLSTM

gaze360 = Gaze360('./gaze360_model.pth.tar', 640, 360)

In [17]:
v_reader = imageio.get_reader('./pokemao.mp4')
fps = v_reader.get_meta_data()['fps']
out = imageio.get_writer('video_gaze.mp4',fps=fps)
color_encoding = []
for i in range(1000): color_encoding.append([random.randint(0,254),random.randint(0,254),random.randint(0,254)])

W = max(int(fps//8),1)

In [18]:
i = 0
v = []
try:
  for frame in v_reader:
    v.append(frame)
except:
  pass  

In [19]:
def print_gaze_in_head(xy, text, img):
  font      = cv2.FONT_HERSHEY_SIMPLEX
  fontScale = 0.5
  fontColor = (0,255,0)
  lineType  = 2

  cv2.putText(img, text, 
    (xy[0],xy[1]), 
    font, 
    fontScale,
    fontColor,
    lineType)

In [20]:
def process_video(frames):
    for i in range(0,len(frames)):
        image = v[i].copy()
        image = cv2.resize(image,(gaze360.width,gaze360.height))
        image = image.astype(float)

        if i in headInFrames:
            people_atention = dict()
            for id_t in headInFrames[i].keys():
                input_image = torch.zeros(7,3,224,224)
                count = 0
                for j in range(i-3*W,i+4*W,W):
                    if j in headInFrames and id_t in headInFrames[j]:
                        new_im = Image.fromarray(frames[j],'RGB')
                        person = headInFrames[j][id_t]
                        bbox,eyes = (person.head, person.eyes)
                    else:
                        new_im = Image.fromarray(frames[i],'RGB')
                        person = headInFrames[i][id_t]
                        bbox,eyes = (person.head, person.eyes)
                    new_im = new_im.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
                    input_image[count,:,:,:] = image_normalize(transforms.ToTensor()(transforms.Resize((224,224))(new_im)))
                    count = count+1

                person = headInFrames[i][id_t]
                bbox,eyes = (person.head, person.eyes)
                
                bbox = np.asarray(bbox).astype(int)

                output_gaze,_ = gaze360.model(input_image.view(1,7,3,224,224).cuda())
                gaze = gaze360.spherical2cartesial(output_gaze).detach().numpy()

                eyes = np.asarray(eyes).astype(float)
                eyes[0],eyes[1] = eyes[0]/float(frames[i].shape[1]),eyes[1]/float(frames[i].shape[0])

                gaze = gaze.reshape((-1))           

                img_arrow = gaze360.render_frame(2*eyes[0]-1,-2*eyes[1]+1,-gaze[0],gaze[1],-gaze[2],0.05)

                binary_img = ((img_arrow[:,:,0]+img_arrow[:,:,1]+img_arrow[:,:,2])==0.0).astype(float)
                binary_img = np.reshape(binary_img,(gaze360.height,gaze360.width,1))
                binary_img = np.concatenate((binary_img,binary_img,binary_img), axis=2)

                image = binary_img*image + img_arrow*(1-binary_img)
                image = image.astype(np.uint8)

                bbox[0],bbox[2] = gaze360.width*bbox[0]/frames[i].shape[1],gaze360.width*bbox[2]/frames[i].shape[1]
                bbox[1],bbox[3] = gaze360.height*bbox[1]/frames[i].shape[0],gaze360.height*bbox[3]/frames[i].shape[0]

                print_gaze_in_head((bbox[0],bbox[1]), 'x: {0:.2f}'.format(gaze[0]*100), image)
                print_gaze_in_head((bbox[0],bbox[1]+50), 'y: {0:.2f}'.format(gaze[1]*100), image)
                print_gaze_in_head((bbox[0],bbox[1]+100), 'z: {0:.2f}'.format(gaze[2]*100), image)
                print_gaze_in_head((bbox[0],bbox[1]+150), 'id_t: {}'.format(id_t), image)
                

                image = cv2.rectangle(image, (bbox[0],bbox[1]), (bbox[2],bbox[3]),color_encoding[min(id_t,900)])
                image = image.astype(float)
                
                people_atention[id_t] = gaze
            print(people_atention)
        image = image.astype(np.uint8)

        out.append_data(image)
    out.close()


In [21]:
process_video(v)

{1: array([-0.78098905,  0.07102893, -0.6204925 ], dtype=float32)}
{1: array([-1.8784527e-01, -2.2275561e-04, -9.8219866e-01], dtype=float32)}
{1: array([ 0.16280353, -0.05661326, -0.9850329 ], dtype=float32)}
{1: array([-0.34090745, -0.01284472, -0.9400091 ], dtype=float32)}
{1: array([-0.08520792, -0.10470331, -0.9908465 ], dtype=float32)}
{1: array([ 0.19044225, -0.12632252, -0.973537  ], dtype=float32)}
{1: array([-0.31970808, -0.0466047 , -0.94636923], dtype=float32)}
{1: array([-0.17066061, -0.1740972 , -0.96982735], dtype=float32)}
{1: array([ 0.23295707, -0.19412847, -0.952914  ], dtype=float32)}
{1: array([-0.05228458, -0.1411085 , -0.9886125 ], dtype=float32)}
{1: array([-0.00296511, -0.26439664, -0.9644095 ], dtype=float32)}
{1: array([ 0.30294913, -0.2562002 , -0.91792333], dtype=float32)}
{1: array([ 0.14212461, -0.22695652, -0.9634788 ], dtype=float32)}
{1: array([ 0.08279356, -0.32757157, -0.94119185], dtype=float32)}
{1: array([ 0.36025   , -0.29572183, -0.8847421 ], dt

{12: array([ 0.39184806, -0.46874475, -0.791665  ], dtype=float32)}
{12: array([ 0.50158435, -0.48377687, -0.7171981 ], dtype=float32)}
{12: array([ 0.37847748, -0.5335733 , -0.75634265], dtype=float32)}
{12: array([ 0.50900155, -0.5514405 , -0.6609318 ], dtype=float32)}
{12: array([ 0.6119284 , -0.5145076 , -0.60068756], dtype=float32)}
{12: array([ 0.56882   , -0.520453  , -0.63684577], dtype=float32)}
{12: array([ 0.60535395, -0.566578  , -0.55904925], dtype=float32)}
{12: array([ 0.64482236, -0.5501938 , -0.53055716], dtype=float32)}
{12: array([ 0.5944582 , -0.55815816, -0.57886   ], dtype=float32)}
{12: array([ 0.63317424, -0.5845215 , -0.50737065], dtype=float32)}
{12: array([ 0.6372049, -0.5691922, -0.5196059], dtype=float32)}
{12: array([ 0.5632697 , -0.57921904, -0.58926445], dtype=float32)}
{12: array([ 0.6652186 , -0.5884031 , -0.45963684], dtype=float32)}
{12: array([ 0.59333104, -0.59770983, -0.5391672 ], dtype=float32)}
{12: array([ 0.5458075, -0.5856794, -0.5992277], dt

{14: array([-0.6684176 , -0.34776568, -0.65747774], dtype=float32), 13: array([-0.38534844,  0.06034689, -0.92079574], dtype=float32), 12: array([-0.7373294 , -0.10394333, -0.6674887 ], dtype=float32)}
{14: array([-0.7074784 , -0.3462314 , -0.61611533], dtype=float32), 13: array([-0.3584611 ,  0.07703085, -0.93036115], dtype=float32), 12: array([-0.7073212 , -0.04939201, -0.7051646 ], dtype=float32)}
{14: array([-0.70635754, -0.3710326 , -0.6028216 ], dtype=float32), 13: array([-0.33774742,  0.07465754, -0.9382712 ], dtype=float32), 12: array([-0.7066826 , -0.03952545, -0.7064258 ], dtype=float32)}
{14: array([-0.65592533, -0.3712553 , -0.65721494], dtype=float32), 13: array([-0.39607975,  0.08432999, -0.9143355 ], dtype=float32), 12: array([-0.7460096 , -0.04858775, -0.6641603 ], dtype=float32)}
{13: array([-0.37130865,  0.08159468, -0.9249174 ], dtype=float32), 14: array([-0.6995282 , -0.37528297, -0.60813075], dtype=float32), 12: array([-0.73798966,  0.00797204, -0.674765  ], dtype=

{13: array([ 0.32617393, -0.1710044 , -0.929714  ], dtype=float32), 15: array([-0.57916397, -0.4050555 , -0.7074597 ], dtype=float32), 16: array([-0.79147106, -0.0726168 , -0.60687757], dtype=float32)}
{13: array([ 0.38023353, -0.17678154, -0.9078385 ], dtype=float32), 15: array([-0.5632033 , -0.36836457, -0.7396686 ], dtype=float32), 16: array([-0.72828376, -0.03207424, -0.68452466], dtype=float32)}
{13: array([ 0.54081327, -0.2374532 , -0.80693054], dtype=float32), 15: array([-0.5663302 , -0.28733626, -0.77246875], dtype=float32), 16: array([-0.6596194 , -0.07193392, -0.7481495 ], dtype=float32)}
{13: array([ 0.513728 , -0.2549868, -0.8191857], dtype=float32), 15: array([-0.5446501 , -0.29708087, -0.7842826 ], dtype=float32), 16: array([-0.84012145, -0.09512173, -0.53399223], dtype=float32)}
{13: array([ 0.56794846, -0.26646996, -0.7787351 ], dtype=float32), 15: array([-0.49451762, -0.29466268, -0.8176957 ], dtype=float32), 16: array([-0.7983139 , -0.04080205, -0.60085773], dtype=flo

{17: array([-0.25218165, -0.00621884, -0.96765995], dtype=float32), 16: array([ 0.7246874 , -0.20768319, -0.6570357 ], dtype=float32)}
{17: array([-0.30250573,  0.01769418, -0.95298326], dtype=float32), 16: array([ 0.6835796 , -0.29770067, -0.66640323], dtype=float32)}
{17: array([-0.2992461 ,  0.04158818, -0.9532692 ], dtype=float32), 16: array([ 0.7274225 , -0.2493722 , -0.63927305], dtype=float32)}
{17: array([-0.29506055,  0.04595591, -0.9543727 ], dtype=float32), 16: array([ 0.7179955 , -0.22286364, -0.6594044 ], dtype=float32)}
{16: array([ 0.6690489 , -0.30633947, -0.6771482 ], dtype=float32)}
{16: array([ 0.7392984 , -0.26998222, -0.6168853 ], dtype=float32)}
{16: array([ 0.69911665, -0.23775601, -0.67432046], dtype=float32)}
{16: array([ 0.69153005, -0.43897498, -0.57366127], dtype=float32)}
{16: array([ 0.69647527, -0.29342088, -0.6548483 ], dtype=float32)}
{16: array([ 0.60921913, -0.25719535, -0.750135  ], dtype=float32)}
{16: array([ 0.6876936 , -0.34989148, -0.6361237 ], 

...

{22: array([-0.30987078,  0.13039242, -0.94179505], dtype=float32), 20: array([-0.6865619, -0.1995447, -0.6991529], dtype=float32), 21: array([-0.8521212 , -0.05040283, -0.52091175], dtype=float32)}
{22: array([-0.29533347,  0.10159754, -0.9499769 ], dtype=float32), 20: array([-0.6485539 , -0.24976735, -0.71902305], dtype=float32), 21: array([-0.8911898 , -0.12395453, -0.4363668 ], dtype=float32)}
{22: array([-0.29082808,  0.10528281, -0.9509651 ], dtype=float32), 20: array([-0.689199  , -0.16746446, -0.7049542 ], dtype=float32), 21: array([-0.9028597 , -0.26813695, -0.33607593], dtype=float32)}
{22: array([-0.30750334,  0.11741345, -0.94427526], dtype=float32), 20: array([-0.6909893 , -0.16941139, -0.70273286], dtype=float32), 21: array([-0.86327195, -0.06980906, -0.49988827], dtype=float32)}
{22: array([-0.32162362,  0.06201383, -0.9448347 ], dtype=float32), 20: array([-0.64890224, -0.24295656, -0.7210396 ], dtype=float32)}
{22: array([-0.31450224,  0.070517  , -0.9466338 ], dtype=fl

{25: array([-0.25980794, -0.04425188, -0.9646458 ], dtype=float32), 26: array([-0.64805406, -0.40367985, -0.6458084 ], dtype=float32), 28: array([-0.608487  ,  0.08584092, -0.7889074 ], dtype=float32)}
{25: array([-0.26431644, -0.03941815, -0.96363014], dtype=float32), 26: array([-0.63952076, -0.4550239 , -0.6196502 ], dtype=float32), 28: array([-0.50038755, -0.09571281, -0.86049485], dtype=float32)}
{25: array([-0.04589904, -0.11388291, -0.99243337], dtype=float32), 26: array([-0.6179118, -0.45744  , -0.6394792], dtype=float32), 28: array([-0.21205524, -0.1620176 , -0.96373385], dtype=float32)}
{25: array([-0.25367865, -0.04580515, -0.96620345], dtype=float32), 26: array([-0.6187823, -0.3874021, -0.6833946], dtype=float32), 28: array([-0.28428993,  0.0319999 , -0.9582042 ], dtype=float32)}
{26: array([-0.59179175, -0.43365994, -0.679501  ], dtype=float32), 25: array([-0.25216803, -0.04439488, -0.9666646 ], dtype=float32), 28: array([-0.18922742, -0.12621073, -0.9737884 ], dtype=float3

{25: array([-0.3427254 ,  0.04968111, -0.938121  ], dtype=float32), 26: array([-0.6639852 , -0.60208774, -0.44341177], dtype=float32), 31: array([-0.84214693,  0.14896536, -0.51826423], dtype=float32)}
{25: array([-0.3362706 ,  0.04727643, -0.940578  ], dtype=float32), 31: array([-0.8896028 ,  0.08449475, -0.4488513 ], dtype=float32), 26: array([-0.66766953, -0.6518708 , -0.3595579 ], dtype=float32)}
{25: array([-0.34464714,  0.04797398, -0.9375056 ], dtype=float32), 26: array([-0.6451051, -0.6637731, -0.3784767], dtype=float32), 31: array([-0.90595156,  0.11115848, -0.40852848], dtype=float32)}
{25: array([-0.34466827,  0.03219459, -0.9381723 ], dtype=float32), 26: array([-0.6653922, -0.5744233, -0.4767505], dtype=float32), 31: array([-0.88483137,  0.15748882, -0.43848675], dtype=float32)}
{25: array([-0.35853702,  0.02705376, -0.9331234 ], dtype=float32), 26: array([-0.67593014, -0.6057024 , -0.4198132 ], dtype=float32), 31: array([-0.8647973 ,  0.08698719, -0.49452898], dtype=float3

{32: array([-0.2748933 , -0.00753224, -0.9614452 ], dtype=float32), 26: array([-0.8487867 , -0.02980096, -0.5278948 ], dtype=float32), 33: array([-0.66844463, -0.09475572, -0.7377013 ], dtype=float32)}
{32: array([-0.2883724 , -0.02122265, -0.95728314], dtype=float32), 26: array([-0.86049813, -0.00347109, -0.5094418 ], dtype=float32), 33: array([-0.35241187,  0.01181465, -0.9357704 ], dtype=float32)}
{26: array([-0.85179764,  0.06389902, -0.51995933], dtype=float32), 32: array([-0.16688775, -0.06745486, -0.98366576], dtype=float32), 33: array([-0.66544217, -0.01078024, -0.7463715 ], dtype=float32)}
{32: array([-0.26440787, -0.06645459, -0.9621186 ], dtype=float32), 26: array([-0.84286606,  0.04985291, -0.5358092 ], dtype=float32)}
{32: array([-0.28124538, -0.07791775, -0.9564674 ], dtype=float32), 34: array([-0.5929206 , -0.00372161, -0.8052523 ], dtype=float32)}
{32: array([-0.1778236, -0.1114908, -0.9777262], dtype=float32), 34: array([-0.5676931 ,  0.02392913, -0.8228924 ], dtype=fl

{35: array([-0.7867237 , -0.40324268, -0.46739823], dtype=float32)}
{35: array([-0.7888887 , -0.29952925, -0.53659755], dtype=float32)}
{35: array([-0.7979518, -0.259856 , -0.543827 ], dtype=float32)}
{35: array([-0.80848145, -0.38735604, -0.44307232], dtype=float32)}
{35: array([-0.79329604, -0.26186278, -0.5496447 ], dtype=float32)}
{35: array([-0.84377664, -0.39921883, -0.35869965], dtype=float32)}
{35: array([-0.82931423, -0.39076418, -0.39942616], dtype=float32)}
{35: array([-0.8436976 , -0.27878106, -0.45875427], dtype=float32)}
{35: array([-0.7240889 , -0.31420434, -0.6139795 ], dtype=float32)}
{35: array([-0.5948886 , -0.18056348, -0.7832652 ], dtype=float32)}


In [22]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('video_gaze.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)
